<h2> lang </h2>


In [ ]:
import pandas as pd

train_data = pd.read_csv("02_train.csv")
test_data = pd.read_csv("02_test_x.csv")

print(train_data.head())
print(test_data.head())

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['reqmsg'])

train_sequences = tokenizer.texts_to_sequences(train_data['reqmsg'])

MAX_LENGTH = max([len(s) for s in train_sequences])
train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

In [ ]:
print(train_padded[0])

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['act_category'])

print(train_labels[:5])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

VOCAB_SIZE = len(tokenizer.word_index) + 1
EMBED_DIM = 128
LSTM_OUT = 64
LABEL_COUNT = len(train_data['act_category'].unique())

# 모델 구조 정의
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_DIM, input_length=MAX_LENGTH))
model.add(LSTM(LSTM_OUT))
model.add(Dense(LABEL_COUNT, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(train_padded, train_labels, batch_size=32, epochs=5)

# 모델 저장
model.save('')

In [ ]:
test_data = pd.read_csv('02_test_x.csv')

test_sequences = tokenizer.texts_to_sequences(test_data['reqmsg'])
test_padded = pad_sequences(test_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

# 예측
predictions = model.predict(test_padded)
predicted_labels = label_encoder.inverse_transform([tf.argmax(pred, axis=-1).numpy() for pred in predictions])

test_data['act_category'] = predicted_labels

test_data.to_csv('', index=False)